In [1]:
import numpy as np
import pandas as pd

# Load data

In [2]:
# load timestamp files
body_ts = np.load('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c_body_times_scaled.npy')
left_ts = np.load('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c_left_times_scaled.npy')
right_ts = np.load('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c_right_times_scaled.npy')

In [3]:
# Load csv files
dlc_right = pd.read_csv('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c_right_dlc_scaled.csv')
dlc_left = pd.read_csv('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c_left_dlc_scaled.csv')
# dlc_body = pd.read_csv('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c_body_dlc_scaled.csv')
trials = pd.read_csv('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c_trials.csv',index_col=0)

In [4]:
# Load wheel
wheel = np.load('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c_right_wheel_scaled.npy')

# Re-indexing
We're going to re-index everything to be matched to the *right* video timestamps. So those timestamps will be in real-time but the body/left video will just tell you which frame corresponds to the left video frame.

(1) re-index videos, dlc, and wheel
(2) re-index trial events

In [5]:
# 1 re-index videos, dlc, and wheel into one file
indexes = pd.DataFrame(columns=['right_ts','left_idx','body_idx',
                                'cr_paw_l_x','cr_paw_l_y','cr_paw_r_x','cr_paw_r_y',
                                'cl_paw_l_x','cl_paw_l_y','cl_paw_r_x','cl_paw_r_y',
                                'wheel'])

In [6]:
def get_dlc_coords(row):
    paw_l_x = row['paw_l_x']
    if np.isnan(paw_l_x):
        paw_l_x = -1
    paw_l_y = row['paw_l_y']
    if np.isnan(paw_l_y):
        paw_l_y = -1
    paw_r_x = row['paw_r_x']
    if np.isnan(paw_r_x):
        paw_r_x = -1
    paw_r_y = row['paw_r_y']
    if np.isnan(paw_r_y):
        paw_r_y = -1
    return (paw_l_x, paw_l_y, paw_r_x, paw_r_y)
    

In [7]:
dlc_left

,Unnamed: 0,nose_tip_x,nose_tip_y,nose_tip_likelihood,pupil_top_r_x,pupil_top_r_y,pupil_top_r_likelihood,pupil_right_r_x,pupil_right_r_y,pupil_right_r_likelihood,...,tube_top_likelihood,tube_bottom_x,tube_bottom_y,tube_bottom_likelihood,tongue_end_l_x,tongue_end_l_y,tongue_end_l_likelihood,tongue_end_r_x,tongue_end_r_y,tongue_end_r_likelihood
0,0,12.363330,16.728505,1.000000,25.918864,10.248203,0.998869,26.297532,10.479257,0.999691,...,0.999935,18.494060,23.802295,0.999977,NaN,NaN,0.000008,NaN,NaN,0.000008
1,1,12.427202,16.761025,1.000000,25.918864,10.250636,0.998243,26.297532,10.479257,0.999018,...,0.999949,18.497523,23.811363,0.999992,NaN,NaN,0.000007,NaN,NaN,0.000008
2,2,12.561496,16.662639,1.000000,25.879392,10.242951,0.998751,26.280684,10.463439,0.999347,...,0.999914,18.497523,23.829553,0.999982,NaN,NaN,0.000512,NaN,NaN,0.001340
3,3,12.566502,16.503333,1.000000,25.891112,10.220261,0.998240,26.283187,10.463439,0.999526,...,0.999964,18.493258,23.815124,0.999977,NaN,NaN,0.000005,NaN,NaN,0.000010
4,4,12.726176,16.121722,1.000000,25.917004,10.201366,0.998754,26.300668,10.454806,0.999514,...,0.999984,18.493258,23.815124,0.999991,NaN,NaN,0.000010,NaN,NaN,0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115705,115705,12.267591,15.886920,1.000000,25.347968,10.258028,0.999955,25.734679,10.497567,0.999821,...,0.999917,18.585687,24.386583,0.999998,NaN,NaN,0.000024,NaN,NaN,0.001836
115706,115706,12.397828,15.704704,1.000000,25.347968,10.298466,0.999946,25.734679,10.512613,0.999734,...,0.999919,18.581711,24.380051,0.999996,NaN,NaN,0.000055,NaN,NaN,0.000034
115707,115707,12.431557,15.704704,0.979132,25.313420,10.304303,0.999953,25.734679,10.539640,0.999918,...,0.999950,18.587051,24.400862,0.999999,NaN,NaN,0.000009,NaN,NaN,0.000013
115708,115708,12.363026,15.757501,1.000000,25.344751,10.239513,0.999870,25.745321,10.502518,0.999471,...,0.999942,18.587051,24.404511,0.999998,NaN,NaN,0.000013,NaN,NaN,0.000013


In [10]:
for idx in np.arange(len(right_ts)):
    rts = right_ts[idx]
    
    # get indexes
    left_idx = np.argmin(np.abs(left_ts-rts))
    body_idx = np.argmin(np.abs(body_ts-rts))
    
    # right camera DLC
    (cr_paw_l_x, cr_paw_l_y, cr_paw_r_x, cr_paw_r_y) = get_dlc_coords(dlc_right.iloc[idx])
    (cl_paw_l_x, cl_paw_l_y, cl_paw_r_x, cl_paw_r_y) = get_dlc_coords(dlc_left.iloc[left_idx])  
    
    indexes.loc[idx] = [rts, left_idx, body_idx,
                        cr_paw_l_x, cr_paw_l_y, cr_paw_r_x, cr_paw_r_y,
                        cl_paw_l_x, cl_paw_l_y, cl_paw_r_x, cl_paw_r_y,
                         wheel[idx]]

In [11]:
indexes

,right_ts,left_idx,body_idx,cr_paw_l_x,cr_paw_l_y,cr_paw_r_x,cr_paw_r_y,cl_paw_l_x,cl_paw_l_y,cl_paw_r_x,cl_paw_r_y,wheel
0,42.899210,0.0,0.0,44.063354,30.317441,42.808729,33.064049,16.926058,31.370781,26.774223,29.237350,-7.272619
1,42.952371,1.0,0.0,41.898382,30.278808,40.316036,35.545930,16.926058,31.370781,27.699503,30.557193,-7.413900
2,42.985634,2.0,1.0,42.989555,28.250240,37.440788,37.092824,16.545179,29.035404,32.215511,30.152882,-7.616598
3,43.018864,3.0,2.0,42.387182,27.445074,40.455170,31.425919,16.658760,29.035404,31.601608,30.000830,-7.824069
4,43.052094,4.0,3.0,40.085285,29.398140,37.380833,35.718389,16.658760,29.944198,30.314884,29.544570,-8.028664
...,...,...,...,...,...,...,...,...,...,...,...,...
115845,3894.032091,115706.0,116103.0,45.093773,30.074131,40.808434,31.888142,21.314013,30.321954,31.321676,34.018535,53.420831
115846,3894.065321,115707.0,116104.0,45.300791,30.044463,40.880482,31.182975,22.126221,30.466529,32.246765,34.125905,53.459022
115847,3894.098551,115708.0,116105.0,45.404114,30.005652,40.886089,31.068314,22.126221,30.459948,32.284637,34.157310,53.467147
115848,3894.131815,115709.0,116106.0,45.452904,29.992549,40.829899,31.092772,22.064081,30.428780,32.157963,34.157310,53.456397


In [15]:
indexes['right_ts'].values.astype(float)

array([  42.89920965,   42.952371  ,   42.98563434, ..., 3894.0985512 ,
       3894.13181454, 3894.16504455])

In [18]:
for key in indexes.keys():
    dat = indexes[key].values.astype(np.float32)
    dat_bytes = dat.tobytes()
    with open('0802ced5-33a3-405e-8336-b65ebc5cb07c/0802ced5-33a3-405e-8336-b65ebc5cb07c.'+key+'.bytes', 'wb') as file:
        file.write(dat_bytes)

In [82]:
indexes.to_csv('0802ced5-33a3-405e-8336-b65ebc5cb07c.timestamps.csv')

In [51]:
trials

,start_timestamp,stim_on_timestamp,feedback_timestamp,stimulus_side,contrast,feedback
0,31.277572,45.118421,45.700196,R,1.000,1.0
1,47.886711,48.453154,48.782721,L,0.250,1.0
2,50.954937,51.502883,51.860947,R,0.250,1.0
3,54.061793,56.286237,56.915608,R,0.125,1.0
4,59.090824,61.585941,61.897343,R,0.250,1.0
...,...,...,...,...,...,...
419,2516.027339,2516.627579,2516.716403,L,0.250,-1.0
420,2520.360605,2520.992409,2529.540654,L,0.125,-1.0
421,2533.177524,2534.674707,2537.883253,L,0.125,-1.0
422,2541.491992,2542.175924,2548.352839,L,1.000,-1.0


In [8]:
# 2 re-index the trial events
for i, row in trials.iterrows():
    # replace the timestamps with the index
    start_idx = np.argmin(np.abs(row['start_timestamp']-right_ts))
    stim_on_idx = np.argmin(np.abs(row['stim_on_timestamp']-right_ts))
    feedback_idx = np.argmin(np.abs(row['feedback_timestamp']-right_ts))
    trials.loc[i,'start_timestamp'] = start_idx
    trials.loc[i,'stim_on_timestamp'] = stim_on_idx
    trials.loc[i,'feedback_timestamp'] = feedback_idx

In [9]:
trials

,start_timestamp,stim_on_timestamp,feedback_timestamp,stimulus_side,contrast,feedback
0,0.0,66.0,84.0,R,1.000,1.0
1,149.0,166.0,176.0,L,0.250,1.0
2,242.0,258.0,269.0,R,0.250,1.0
3,335.0,402.0,421.0,R,0.125,1.0
4,486.0,562.0,571.0,R,0.250,1.0
...,...,...,...,...,...,...
419,74394.0,74412.0,74414.0,L,0.250,-1.0
420,74524.0,74543.0,74800.0,L,0.125,-1.0
421,74910.0,74955.0,75051.0,L,0.125,-1.0
422,75160.0,75180.0,75366.0,L,1.000,-1.0


In [54]:
trials.to_csv('0802ced5-33a3-405e-8336-b65ebc5cb07c.trials.csv')